In [49]:
from pymongo import MongoClient
from bson.son import SON
import datetime
import pandas as pd


In [4]:
db = MongoClient().texts.dail

## Group Dáil contributions by month and year.

In [56]:
speeches_by_month = db.aggregate(
  [ 
    { "$project" : { "year": {"$year": "$date"} ,
                    "month" : { "$month" : "$date" },
                   "word_count" : "$len_doc"} } ,
    { "$group" : { "_id" : {"month":"$month",
                            "year":"$year"} ,
                  "contributions_count" : { "$sum" : 1 },
                  "total_words": {"$sum": "$word_count"},
                    "avg_word_count": {"$avg" : "$word_count"}} },
    { "$sort" : { "_id.year": 1,
                 "_id.month" : 1 } }
  ]
)
speeches = [s for s in speeches_by_month]

In [65]:
#flatten _id
for s in speeches:
    s['year'] = s["_id"]['year']
    s['month'] = s["_id"]['month']
    s.pop("_id")

In [66]:
counts = pd.DataFrame(speeches)

In [69]:
counts.to_pickle("dail_contributions.pickle")